In [ ]:
from moviepy.editor import VideoFileClip, AudioFileClip
import glob
import pandas as pd
import re
import os
from IPython.display import Video, Audio

In [ ]:
# Visualization of VAS_Stress
import matplotlib.pyplot as plt

# Set the figure size
plt.rcParams['figure.figsize'] = [10, 5]

# Define the labels for the x-axis
new_dst_labels = ["T0", "T1", "T2a", "T2", "T3", "T4", "T5"]
new_tsst_labels = ["T0", "T1", "T2", "T3", "T4", "T5"]

# Create the subplots grid
fig, axs = plt.subplots(1, 2, sharey=True)

# Plot the first figure on the left subplot
participants.filter(like="dst_vas_stress_T").T.plot(legend=False, ax=axs[0])
axs[0].set_xticks(range(len(new_dst_labels)))
axs[0].set_xticklabels(new_dst_labels)
axs[0].set_title('DST')

# Plot the second figure on the right subplot
participants.filter(like="tsst_vas_stress_T").T.plot(legend=False, ax=axs[1])
axs[1].set_xticks(range(len(new_tsst_labels)))
axs[1].set_xticklabels(new_tsst_labels)
axs[1].set_title('TSST')

# Set the y-axis label
fig.text(0.04, 0.5, 'VAS stress', va='center', rotation='vertical')

# Set the overall title
fig.suptitle('VAS stress for DST and TSST')

# Show the plot
plt.show()

In [ ]:
input_file = video_files_tsst[3]
output_file= "/data/dst_tsst_22_bi_multi_nt_lab/processed/audio_files/" + input_file.rsplit('/', 1)[1][:-3] + "mp3"
print(input_file, output_file)


In [ ]:
audio_files_tsst = glob.glob("/data/dst_tsst_22_bi_multi_nt_lab/processed/audio_files/**.mp3",recursive=True)
print(audio_files_tsst)

In [ ]:


print(audio_files_tsst[0])

In [ ]:
input_file = VideoFileClip(video_files_tsst2[1])
output_file= "/data/dst_tsst_22_bi_multi_nt_lab/processed/audio_files/test_TSST2_audio_quality.mp3"
# print(input_file, output_file)
# skip creating mp3, if already exists
if not os.path.exists(output_file):
    clip = VideoFileClip(input_file)
    clip.audio.write_audiofile(output_file)
Audio(output_file)

In [ ]:
print(len(audio_files_tsst))

In [ ]:
print(audio_files_tsst)

In [ ]:
from mutagen.mp3 import MP3

# Provide the path to your MP3 file
file_path = '/path/to/your/file.mp3'

# Load the MP3 file using Mutagen
mp3 = MP3(file_path)

# Check the encoding
encoding = mp3.info.encoding
print("MP3 Encoding:", encoding)